# Bank Marketing Classification

## Data Collection/Preprocessing

Person A
- Load data
- Feature analysis (check for imbalanced data, heatmap feature correlation, etc.)

Person B
- Feature engineering (feature scaling, normalization, standardization, one hot encode features/targets, check for missing values)
- Train test validation split

In [ ]:
# import the dataset -- can use pip install

# pip install ucimlrepo
# from ucimlrepo import fetch_ucirepo 
  
# # fetch dataset 
# bank_marketing = fetch_ucirepo(id=222) 
  
# # data (as pandas dataframes) 
# X = bank_marketing.data.features 
# y = bank_marketing.data.targets 
  
# # metadata 
# print(bank_marketing.metadata) 
  
# # variable information 
# print(bank_marketing.variables) 

## Baselines

Person C
- Logistic regression

Person D

- Random guessing

## Training Model

Options:

Person C
- Ensemble (random forest, xg boost)

Person D
- Simple neural network

## Evaluating Model

Person E
- Metric 1: F1 Score
- Metric 2: Area Under Receiver Operating Characteristics Curve (ROC)
- Also responsible for conclusion in report